In [130]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from tqdm.notebook import trange
import itertools as it
from sklearn.metrics import roc_auc_score

In [7]:
from forest import *

In [119]:
class WeakRandomForest:
    def __init__(self, trees=1000, max_depth=7, alpha=1.0, beta=None, bootstrap=True, decision='avg'):
        self.trees = trees
        self.max_depth = max_depth
        self.alpha = alpha
        self.beta = beta
        self.bootstrap = bootstrap
        self.decision = decision
        self._tree_array = []
        
    def fit(self, X, y):
        for _ in trange(self.trees):
            tree = WeakTree(X, y, alpha=self.alpha, beta=self.beta, bootstrap=self.bootstrap, 
                                decision=self.decision, max_depth=self.max_depth)
            self._tree_array.append(tree)
    
    def predict(self, x):
        out = np.empty((self.trees))
        for i in range(self.trees):
            out[i] = self._tree_array[i].predict(x)
        return sum(out) / len(out)

In [120]:
class WeakTree:
    def __init__(self, X, y=None, alpha=1.0, beta=None, bootstrap=True, decision='avg', max_depth=20):
        if len(X.shape) != 2:
            raise AttributeError("X must have shape equal to 2")
        n_samples = X.shape[0]
        n_features = X.shape[1]
        if beta is None:
            beta = 1.0 / np.sqrt(n_features)
        cf = self._choose_features(n_features, int(n_features * beta))
        cs = self._choose_samples(n_samples, int(n_samples * alpha), bootstrap=bootstrap)
        X_choice = X[cs, :][:, cf]
        y_choice = None
        if not y is None:
            y_choice = y[cs]
        self.root = WeakTreeNode(max_depth=max_depth, decision=decision)
        self.root.fit(X_choice, y_choice)
    
    def _choose_features(self, n_features, target_n_features):
        choice = np.random.choice(np.arange(n_features), size=target_n_features, replace=False)
        return choice
    
    def _choose_samples(self, n_samples, target_n_samples=None, bootstrap=True):
        if target_n_samples is None:
            target_n_samples = n_samples
        if bootstrap == True:
            if target_n_samples != n_samples:
                raise AttributeError("Target samples number should be equal to the dataset size when bootstrap==True")
        choice = np.random.choice(np.arange(n_samples), size=target_n_samples, replace=bootstrap)
        return choice
    
    def predict(self, x):
        return self.root.predict(x)


In [138]:
class WeakTreeNode:
    def __init__(self, depth=0, max_depth=20, decision='avg'):
        """
        decision in ('avg', 'grid')
        """
        self.left = None
        self.right = None
        self.feature = None
        self.cut = None
        self.depth = depth
        self.max_depth = max_depth
        self.decision = decision
        self.prediction = None
            
    def fit(self, X, y):
#         print('.', end='')
        if self.depth < self.max_depth and X.shape[0] > 1 and np.unique(y).shape[0] > 1:
            grid = self._init_grid(X, y)
            min_loss = math.inf
            min_args = None
            
#             for f_set, c_set in tqdm(grid.items()):
            for f_set, c_set in grid.items():
                cuts1 = c_set[0]
                cuts2 = c_set[1]
                cuts3 = c_set[2]
                for c1 in cuts1:
                    for c2 in cuts2:
                        for c3 in cuts3:
                            loss_val = self.loss(X, y, *f_set, c1, c2, c3)
                            if loss_val < min_loss:
                                min_loss = loss_val
                                min_args = (f_set, (c1, c2, c3))
            
            if min_loss < math.inf:
                self.feature = min_args[0][0]
                self.cut = min_args[1][0]
                self.left = WeakTreeNode(depth=self.depth + 1, max_depth=self.max_depth, decision=self.decision)
                self.right = WeakTreeNode(depth=self.depth + 1, max_depth=self.max_depth, decision=self.decision)
                self.left.feature = min_args[0][1]
                self.left.cut = min_args[1][1]
                self.right.feature = min_args[0][2]
                self.right.cut = min_args[1][2]
                self.create_children(X, y)
            else:
                c = y[y == 1.0].sum()
                self.prediction = c / y.shape[0]
        else:
            c = y[y == 1.0].sum()
            self.prediction = c / y.shape[0]
        
    def _init_grid(self, X, y):
        UNIQUE = 10
        n_features = X.shape[1]
        grid = {}
        f_sets = it.permutations(np.arange(n_features), 3)
        features = {}
        for f_num in range(n_features):
            feature = X[:, f_num]
            if self.decision == 'grid':
                f_min = feature.min()
                f_max = feature.max()
                f_unique = np.unique(feature)
                if len(f_unique) <= UNIQUE:
                    vals = [(f_unique[i] + f_unique[i+1]) / 2.0 for i in range(len(f_unique) - 1)]
                    features[f_num] = vals
                else:
                    features[f_num] = np.arange(f_min, f_max, (f_max - f_min) / UNIQUE)[1:]
            elif self.decision == 'avg':
                features[f_num] = [(feature.max() - feature.min()) / 2.0]
        for f_set in f_sets:
            grid[f_set] = (features[f_set[0]], features[f_set[1]], features[f_set[2]])
        return grid
    
    def loss(self, X, y, f1, f2, f3, c1, c2, c3):
        
        #             X
        #           /   \_
        #          /      \
        #       X_l        X_r
        #      /   \      /   \
        #  X_ll   X_lr  X_rl   X_rr
        
        # First node division
        m1 = X[:, f1] < c1
        X_l = X[m1, :]  # Left X subset
        X_r = X[~m1, :] # Right X subset
        y_l = y[m1]     # Left y subset
        y_r = y[~m1]    # Right y subset
        
        # Left sub-node division
        m2 = X_l[:, f2] < c2
        y_ll = y_l[m2]     # Left-left y subset
        y_lr = y_l[~m2]    # Left-right y subset
        
        # Right sub-node division
        m3 = X_r[:, f3] < c3
        y_rl = y_r[m3]     # Right-left y subset
        y_rr = y_r[~m3]    # Right-right y subset
        
        # Count amount of elements in nodes
        l_size = m1.sum()
        r_size = (~m1).sum()
        ll_size = m2.sum()
        lr_size = (~m2).sum()
        rl_size = m3.sum()
        rr_size = (~m3).sum()
        
        if l_size == 0 or r_size == 0 or ll_size == 0 or lr_size == 0 or rl_size == 0 or rr_size == 0:
            return math.inf
        
        # Amounts of classes in sub-nodes after division
        _, y_llc = np.unique(y_ll, return_counts=True)
        _, y_lrc = np.unique(y_lr, return_counts=True)
        _, y_rlc = np.unique(y_rl, return_counts=True)
        _, y_rrc = np.unique(y_rr, return_counts=True)
        
        # Compute entropy values
        e_ll = entropy(y_llc, ll_size)
        e_lr = entropy(y_lrc, lr_size)
        e_rl = entropy(y_rlc, rl_size)
        e_rr = entropy(y_rrc, rr_size)
        
        return (ll_size / l_size * e_ll) + (lr_size / l_size * e_lr) + (rl_size / r_size * e_rl) + (rr_size / r_size * e_rr)
    
    def create_children(self, X, y):        
        f = self.feature
        c = self.cut
        fl = self.left.feature
        cl = self.left.cut
        fr = self.right.feature
        cr = self.right.cut
        
        # First node division
        m1 = X[:, f] < c
        X_l = X[m1, :]  # Left X subset
        X_r = X[~m1, :] # Right X subset
        y_l = y[m1]     # Left y subset
        y_r = y[~m1]    # Right y subset
        
        # Left sub-node division
        m2 = X_l[:, fl] < cl
        X_ll = X_l[m2, :]
        X_lr = X_l[~m2, :]
        y_ll = y_l[m2]
        y_lr = y_l[~m2]
        
        # Right sub-node division
        m3 = X_r[:, fr] < cr
        X_rl = X_r[m3, :]
        X_rr = X_r[~m3, :]
        y_rl = y_r[m3]
        y_rr = y_r[~m3]
        
        self.left.left = WeakTreeNode(self.depth + 2, decision=self.decision, max_depth=self.max_depth)
        self.left.right = WeakTreeNode(self.depth + 2, decision=self.decision, max_depth=self.max_depth)
        self.right.left = WeakTreeNode(self.depth + 2, decision=self.decision, max_depth=self.max_depth)
        self.right.right = WeakTreeNode(self.depth + 2, decision=self.decision, max_depth=self.max_depth)
        self.left.left.fit(X_ll, y_ll)
        self.left.right.fit(X_lr, y_lr)
        self.right.left.fit(X_rl, y_rl)
        self.right.right.fit(X_rr, y_rr)
    
    def predict(self, x):
        if self.prediction is None:
            mask = x[self.feature] < self.cut
            if mask:
                pred = self.left.predict(x)
            else:
                pred = self.right.predict(x)
        else:
            pred = self.prediction
        return pred
    
def visualize_grid(grid):
    for k, v in grid.items():
        print(k)
        for i in range(3):
            print('   ', k[i], '=>', v[i])
            
def entropy(bucket, total_size):
    probs = bucket / total_size
    return 1.0 - probs.dot(probs)
    

In [142]:
df = pd.read_csv('df_21jan.csv', delimiter=';', decimal=',')

In [190]:
# df = df.drop(df[['region_90', 'region_24', 'region_71', 'region_32', 'region_20','region_53', 'region_69', 'region_4', 'region_5', 'region_18', 'SMN_3','SMN_2', 'EDUC_1', 'EDUC_0']], axis=1)
tr_mask = np.zeros(df.shape[0], dtype=bool)
tr_mask[:tr_n] = True
np.random.shuffle(tr_mask)

X_data = df.drop(df.columns[[0]], axis=1).to_numpy()
y_data = df['AH'].to_numpy()
tr_n = 10000

X_train = X_data[tr_mask]
y_train = y_data[tr_mask]
X_test = X_data[~tr_mask]
y_test = y_data[~tr_mask]

# X_train = X_data[:tr_n]
# y_train = y_data[:tr_n]
# X_test = X_data[tr_n:]
# y_test = y_data[tr_n:]

In [144]:
f = WeakRandomForest(decision='grid', trees=100, max_depth = 1000)
f.fit(X_train, y_train)

  0%|          | 0/100 [00:00<?, ?it/s]

In [146]:
out_train = np.empty(y_train.shape)
for i in trange(X_train.shape[0]):
    out_train[i] = f.predict(X_train[i])
    
out_test = np.empty(y_test.shape)
for i in trange(X_test.shape[0]):
    out_test[i] = f.predict(X_test[i])
    
roc_value = roc_auc_score(y_test, out_test)
print(roc_value)
roc_value = roc_auc_score(y_train, out_train)
print(roc_value)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/3912 [00:00<?, ?it/s]

0.6686282771869934
0.6649113240486189


In [145]:
# 0.6411931067482735
# 0.6509489631079604

In [191]:
auc = []
for i in range(10):
    tr_mask = np.zeros(df.shape[0], dtype=bool)
    tr_mask[:tr_n] = True
    np.random.shuffle(tr_mask)

    X_data = df.drop(df.columns[[0]], axis=1).to_numpy()
    y_data = df['AH'].to_numpy()
    tr_n = 10000

    X_train = X_data[tr_mask]
    y_train = y_data[tr_mask]
    X_test = X_data[~tr_mask]
    y_test = y_data[~tr_mask]
    f = WeakRandomForest(decision='grid', trees=100, max_depth = 1000)
    f.fit(X_train, y_train)
    
    out_train = np.empty(y_train.shape)
    for i in trange(X_train.shape[0]):
        out_train[i] = f.predict(X_train[i])

    out_test = np.empty(y_test.shape)
    for i in trange(X_test.shape[0]):
        out_test[i] = f.predict(X_test[i])

    roc_value_test = roc_auc_score(y_test, out_test)
    print(roc_value_test)
    roc_value_train = roc_auc_score(y_train, out_train)
    print(roc_value_train)
    auc.append((roc_value_test, roc_value_train))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/3912 [00:00<?, ?it/s]

0.7121151002626824
0.7121705863502261


  0%|          | 0/100 [00:00<?, ?it/s]

...

0.7129669045816058
0.7123493034305675


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/3912 [00:00<?, ?it/s]

0.6938727222291506
0.6843703377583095


In [193]:
print(auc)

[(0.7121151002626824, 0.7121705863502261), (0.727785893013427, 0.7235451470945236), (0.6936253846574642, 0.69044742188635), (0.7067830660945498, 0.7085237049611302), (0.6951198983726024, 0.7071829113230442), (0.710221421215242, 0.7133123107896056), (0.7026683778019405, 0.7144175452439443), (0.696972569210612, 0.6866091614779316), (0.7129669045816058, 0.7123493034305675), (0.6938727222291506, 0.6843703377583095)]


In [192]:
print(sum([x[0] for x in auc]) / len(auc))
print(sum([x[1] for x in auc]) / len(auc))

0.7052131337439277
0.7052928430315633


In [149]:
X = np.empty((20, 5))
for i in range(20):
    X[i, 0] = i % 2
    X[i, 1] = 1.0 / 20 * i
    X[i, 2] = i
    X[i, 3] = 1.0 / 20 * (20 - i)
    X[i, 4] = i % 5

In [150]:
pd.DataFrame(X)

,0,1,2,3,4
0,0.0,0.00,0.0,1.00,0.0
1,1.0,0.05,1.0,0.95,1.0
2,0.0,0.10,2.0,0.90,2.0
3,1.0,0.15,3.0,0.85,3.0
4,0.0,0.20,4.0,0.80,4.0
5,1.0,0.25,5.0,0.75,0.0
6,0.0,0.30,6.0,0.70,1.0
7,1.0,0.35,7.0,0.65,2.0
8,0.0,0.40,8.0,0.60,3.0
9,1.0,0.45,9.0,0.55,4.0


In [162]:
np.where(X[np.random.choice(np.arange(X.shape[0]), size=15, replace=False)], True, False)

array([[False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [False,  True,  True,  True, False],
       [ True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [False, False, False,  True, False],
       [False,  True,  True,  True,  True],
       [ True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True],
       [False,  True,  True,  True,  True],
       [ True,  True,  True,  True, False]])

In [ ]:
X[np.random.choice(np.arange(X.shape[0]), size=15, replace=False)]

In [186]:
tr_mask = np.zeros(df.shape[0], dtype=bool)
tr_mask[:tr_n] = True
np.random.shuffle(tr_mask)

In [189]:
a

array([False, False, False, ..., False,  True,  True])